# Calculation of the cross entropy loss (NLL) for a classification tasks


**Goal:** In this notebook you will use Keras to set up a CNN for classification of MNIST images and calculate the cross entropy before the CNN was trained. You will use basic numpy functions to calculate the loss that is expected from random guessing and see that an untrained CNN is not better than guessing.

**Usage:** The idea of the notebook is that you try to understand the provided code by running it, checking the output and playing with it by slightly changing the code and rerunning it.

**Dataset:** You work with the MNIST dataset. You have 60'000 28x28 pixel greyscale images of digits (0-9).

**Content:**
* load the original MNIST data
* define a CNN in Keras
* evaluation of the cross entropy loss function of the untrained CNN for all classes
* implement the loss function yourself using the predicted probabilities and numpy


| [open in colab](https://colab.research.google.com/github/tensorchiefs/dl_book/blob/master/chapter_04/nb_ch04_02.ipynb)



#### Imports

First you load all the required libraries.

In [26]:
# load required libraries:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from sklearn.metrics import confusion_matrix

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten , Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

### Exercise: Likelihood if you have no clue
<img src="https://raw.githubusercontent.com/tensorchiefs/dl_book/master/imgs/paper-pen.png" width="60" align="left" />  
If you have no idea about the training dataset, your guess for every image would be 1/nr_of_classes. Calculate the NLL for that case.

In [27]:
# @title Solution
nr_of_classes=10
-np.log(1/nr_of_classes)

2.3025850929940455

## Likelihood of untrained CNN





#### Loading and preparing the MNIST data


In [39]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_train=x_train #/ 255 #divide by 255 so that they are in range 0 to 1
X_train=np.reshape(X_train, (X_train.shape[0],28,28,1))
Y_train=tensorflow.keras.utils.to_categorical(y_train,10) # one-hot encoding


Y_train.shape, X_train.shape

((60000, 10), (60000, 28, 28, 1))

## CNN model



In [40]:
# here you define hyperparameter of the CNN
batch_size = 128
nb_classes = 10
img_rows, img_cols = 28, 28
kernel_size = (3, 3)
input_shape = (img_rows, img_cols, 1)
pool_size = (2, 2)

In [41]:
# define CNN with 2 convolution blocks and 2 fully connected layers
model = Sequential()

model.add(Convolution2D(8,kernel_size,padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(8, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(16, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(16,kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(40))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# compile model and intitialize weights
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
# summarize model along with number of model weights
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 28, 28, 8)         80        
                                                                 
 activation_18 (Activation)  (None, 28, 28, 8)         0         
                                                                 
 conv2d_13 (Conv2D)          (None, 28, 28, 8)         584       
                                                                 
 activation_19 (Activation)  (None, 28, 28, 8)         0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 14, 14, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 14, 14, 16)        1168      
                                                      

Here you predict the probabilities for all images in the training data set. You did not train the network yet, therefore the probabilities will be around 10% for each class.

In [43]:
# Calculate the probailities for the training data
Pred_prob = model.predict(X_train)

1875/1875 [==============================] - 4s 2ms/step


In [44]:
Pred_prob[0:5]

array([[1.3205596e-18, 5.2988291e-12, 2.5350449e-20, 1.2180353e-22,
        1.8144437e-16, 4.7060991e-21, 1.5764765e-08, 1.5844094e-17,
        5.9404232e-11, 1.0000000e+00],
       [2.7667373e-24, 2.3641273e-11, 2.2651152e-25, 4.4870147e-20,
        3.5441853e-15, 9.4421481e-27, 9.9942786e-01, 2.2959579e-11,
        2.4655371e-09, 5.7209819e-04],
       [1.3653172e-02, 8.3011842e-01, 6.5737753e-03, 1.5207104e-10,
        4.7123001e-14, 3.8285774e-20, 2.3730610e-05, 3.8681562e-12,
        3.6684156e-11, 1.4963086e-01],
       [9.1495855e-14, 1.3210382e-09, 1.6606809e-12, 5.4113473e-16,
        2.6305097e-15, 7.8905446e-23, 9.9965596e-01, 4.8645860e-10,
        4.0532360e-09, 3.4400207e-04],
       [3.3100608e-08, 4.6578123e-12, 3.2831602e-09, 4.1076941e-18,
        8.9987955e-19, 3.1901950e-16, 4.6687070e-03, 8.5858165e-10,
        1.3493828e-05, 9.9531770e-01]], dtype=float32)

In [45]:
Pred_prob.shape, Y_train.shape

((60000, 10), (60000, 10))

### Exercise : Calculate the loss function using numpy
<img src="https://raw.githubusercontent.com/tensorchiefs/dl_book/master/imgs/paper-pen.png" width="60" align="left" />  

*Exercise : Use numpy to calculate the value of the negative log-likelihood loss (=cross entropy) that you expect for the untrained CNN, which you have constructed above to discriminate between the 10 classes. Determine the cross entropy that results from the predicted probabilities (Pred_prob). To determine the cross entropy of the prediction, you can loop over each example and use its true label (Y_train) and the predicted probability for the true class. Do you get the cross entropy value that you have expected?*




In [46]:
# Write your code here

Scroll down to see the solution.

</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>

In the next cell you calculate the cross entropy loss of each single image, then you sum up all individual losses and divide the sum with the nr of training examples. You take the negative of this result to get the NLL, also known as categorical cross entropy.

In [47]:
loss=np.zeros(len(X_train))
Y=np.argmax(Y_train,axis=1)
for i in range(0,len(X_train)):
  loss[i]=np.log(Pred_prob[i][Y[i]])
-np.sum(loss)/len(X_train)

25.79395893874508

You get more a similar result as as you got with the model.evaluate function for the untrained CNN.  

In [48]:
model.evaluate(X_train, Y_train,verbose=2)

1875/1875 - 4s - loss: 25.7940 - accuracy: 0.1081 - 4s/epoch - 2ms/step


[25.793962478637695, 0.1080833300948143]

### Exercise: Change normalization
<img src="https://raw.githubusercontent.com/tensorchiefs/dl_book/master/imgs/paper-pen.png" width="60" align="left" />  

Load the data again, but this time do not scale the data. Repeat the analysis. What is the result? Why is it a good idea to check the loss for untrained networks.

In [ ]:
# Scroll Down for solution





























#

In the unscaled case, you do not necessarily get 1/10 for the probabilities, and thus the loss is usually larger than 2.3. So, the training starts worse than what you would get from pure guessing. This poor start not only leads to a higher initial loss but also increases training time. Effective initialization ensures a smoother and more efficient learning trajectory.